# Finite Markov Decision Processes

The formal problem of ***Finite Markov decision processes***, or ***finite MDPs***, involves evaluative feedback, as in bandits, but also an associative aspect—choosing different actions in different situations. MDPs are a classical formalization of sequential decision making, where actions influence not just immediate rewards, but also subsequent situations, or states, and through those future rewards.

Whereas in bandit problems we estimated the value $q^*(a)$ of each action a, in MDPs we estimate the value $q^*(s,a)$ of each action $a$ in each state $s$, or we estimate the value $v^*(s)$ of each state given optimal action selections. 

## The Agent–Environment Interface

MDPs are meant to be a straightforward framing of the problem of learning from interaction to achieve a goal. The learner and decision maker is called the ***agent***. The thing it interacts with, comprising everything outside the agent, is called the ***environment***. 
These interact continually, the agent selecting actions and the environment responding to these actions and presenting new situations to the agent. The environment also gives rise to rewards, special numerical values that the agent seeks to maximize over time through its choice of actions.

![MDP](https://images.deepai.org/django-summernote/2019-03-19/6e934c07-9ac4-4321-8803-687049be4796.png)

At each time step $t$, the agent receives some representation of the environment’s state, $S_t \in \mathcal{S}$, and on that basis selects an action, $A_t \in \mathcal{A}(s)$. One time step later, in part as a consequence of its action, the agent receives a numerical reward, $R_{t+1} \in \mathcal{R}$, and finds itself in a new state, $S_{t+1}$.

In a finite MDP, the sets of states, actions, and rewards $(\mathcal{S}, \mathcal{A}, \mathcal{R})$ all have a finite number of elements. For particular values of $R_t$ and $S_t$ random variables, $s^{\prime} \in \mathcal{S}$ and $r \in \mathcal{R}$, there is a probability of those values occurring at time t, given particular values of the preceding state and action:


$$ p(s', r | s , a) \space \dot{=} \space Pr\{S_t = s', R_t = r | S_{t-1} = s, A_{t-1} = a \} $$

for all $s', s \in \mathcal{S}$, $r \in \mathcal{R}$, and $a \in \mathcal{A}(s)$. The function $p$ defines the ***dynamics*** of the MDP.

In a _Markov_ decision process, the probabilities given by $p$ completely characterize the environment’s dynamics. The probability of each possible value for $S_t$ and $R_t$ depends on the immediately preceding state and action, $S_{t-1}$ and $A_{t-1}$, and, given them, not at all on earlier states and actions.

From the four-argument dynamics function, p, we can compute the ***state-transition probabilities***, as a three-argument function $p : \mathcal{S} \times \mathcal{S} \times \mathcal{A} \rightarrow [0,1]$:
$$ p(s' | s , a) \space \dot{=} \space Pr\{S_t = s' | S_{t-1} = s, A_{t-1} = a \} = \sum_{r\in\mathcal{R}} p(s', r | s , a) $$

We can also compute the expected rewards for ***state–action*** pairs as a two-argument function $r : \mathcal{S}\times\mathcal{A} \rightarrow \mathbb{R}$:
$$ r(s , a) \space \dot{=} \space \mathbb{E}\{R_t | S_{t-1} = s, A_{t-1} = a \} = \sum_{r\in\mathcal{R}} r \sum_{s' \in \mathcal{s}} p(s', r | s , a) $$

and the expected rewards for ***state–action–next-state*** triples as a three-argument function $r : \mathcal{S}\times\mathcal{A}\times\mathcal{S} \rightarrow \mathbb{R}$:
$$ r(s , a, s') \space \dot{=} \space \mathbb{E}\{R_t | S_{t-1} = s, A_{t-1} = a, S_{t} = s' \} = \sum_{r\in\mathcal{R}} r \frac{p(s', r | s , a)}{p(s' | s, a)} $$

## Goals and Rewards

In reinforcement learning, the purpose or goal of the agent is formalized in terms of a special signal, called the _reward_, passing from the environment to the agent. At each time step, the reward is a simple number, $R_t \in \mathbb{R}$. Informally, the agent’s goal is to maximize the total amount of reward it receives in the long run.

We can clearly state this informal idea as the reward hypothesis:
> That all of what we mean by goals and purposes can be well thought of as the maximization of the expected value of the cumulative sum of a received scalar signal (called reward).

For example, a chess-playing agent should be rewarded only for actually winning, not for achieving subgoals such as taking its opponent’s pieces or aining control of the center of the board. If achieving these sorts of subgoals were rewarded, then the agent might find a way to achieve them without achieving the real goal.

The reward signal is your way of communicating to the agent what you want achieved, not how you want it achieved.

## Returns and Episodes

If the sequence of rewards received after time step $t$ is denoted $R_{t+1}, R_{t+2}, R_{t+3}, ...$, then we seek to maximize the _expected return_, where the return, denoted $G_t$, is defined as some specific function of the reward sequence. In the simplest case the return is the sum of the rewards:
$$ G_t \doteq R_{t+1} + R_{t+2} + R_{t+3} + ... + R_T $$
where T is a final time step.

In some applications, there is a natural notion of final time step, and the agent–environment interaction breaks naturally into subsequences, which we call ***episodes*** (trials). Each episode ends in a special state called the __terminal state__, followed by a reset to a standard starting state or to a sample from a standard distribution of starting states.

According to the ***discounting*** approach, the agent tries to select actions so that the sum of the discounted rewards it receives over the future is maximized. In particular, it chooses At to maximize the expected discounted return:
$$ G_t \doteq R_{t+1} + \gamma R_{t+2} + \gamma^2 R_{t+3} + ... = \sum_{k=0}^\infty \gamma^k R_{t+k+1}$$
where $\gamma$ is a parameter, $0 < \gamma < 1$, called the **discount rate**.

Note that although the return is a sum of an infinite number of terms, it is still finite if the reward is nonzero and constant—if $\gamma < 1$.

If $\gamma$ = 0, the agent is “myopic” in being concerned only with maximizing immediate rewards: its objective in this case is to learn how to choose $A_t$ so as to maximize only $R_{t+1}$.
As $\gamma$ approaches 1, the return objective takes future rewards into account more strongly; the agent becomes more farsighted.

$$\begin{aligned}
G_t &\doteq R_{t+1} + \gamma R_{t+2} + \gamma^2 R_{t+3} + \gamma^3 R_{t+4} + \dots \\
&= R_{t+1} + \gamma \left(R_{t+2} + \gamma R_{t+3} + \gamma^2 R_{t+4} + \dots \right) \\
&= R_{t+1} + \gamma G_{t+1}
\end{aligned}$$

## Example: Pole-Balancing

The objective in this task is to apply forces to a cart moving along a track so as to keep a pole hinged to the cart from falling over: A failure is said to occur if the pole falls past a given angle from vertical or if the cart runs off the track. The pole is reset to vertical after each failure. This task could be treated as episodic, where the natural episodes are the repeated attempts to balance the pole. The reward in this case could be $+1$ for every time step on which failure did not occur, so that the return at each time would be the number of steps until failure. In this case, successful balancing forever would mean a return of infinity. 

Alternatively, we could treat pole-balancing as a continuing task, using discounting. In this case the reward would be $-1$ on each failure and zero at all other times. The return at each time would then be related to $-
\gamma^{K-1}$, where $K$ is the number of time steps before failure (as well as to the times of later failures). In either case, the return is maximized by keeping the pole balanced for as long as possible.

## Policies and Value Functions

Almost all reinforcement learning algorithms involve estimating **value functions**—functions of states (or of state–action pairs) that estimate _how good_ it is for the agent to be in a given state (or how good it is to perform a given action in a given state). Value functions are defined with respect to particular ways of acting, called _policies_.

Formally, a ***policy*** is a mapping from states to probabilities of selecting each possible action. If the agent is following policy $\pi$ at time $t$, then $\pi_t(a|s)$ is the probability that $A_t = a$ if $S_t = s$.

The value function of a state $s$ under a policy $\pi$, denoted $v_\pi(s)$, is the expected return when starting in $s$ and following $\pi$ thereafter. For MDPs, we can define $v_\pi$ formally by
$$ v_\pi(s) \doteq \mathbb{E}_\pi \left[G_t | S_t = s \right] = \mathbb{E}_\pi \left[ \sum_{k=0}^\infty \gamma^k R_{t+k+1} | S_t = s \right] $$
where $\mathbb{E}[\space]$ denotes the expected value of a random variable given that the agent follows policy $\pi$, and $t$ is any time step.

Similarly, we define the value of taking action $a$ in state $s$ under a policy $\pi$, denoted $q_\pi(s,a)$, as the expected return starting from $s$, taking the action $a$, and thereafter following policy $\pi$:
$$ q_\pi(s,a) \doteq \mathbb{E}_\pi \left[G_t | S_t = s, A_t = a \right] = \mathbb{E}_\pi \left[ \sum_{k=0}^\infty \gamma^k R_{t+k+1} | S_t = s, A_t = a \right] $$
We call $q_\pi$ the ***action-value function*** for policy $\pi$.

The value functions $v_\pi$ and $q_\pi$ can be estimated from experience. For example, if an agent follows policy $\pi$ and maintains an average, for each state encountered, of the actual returns that have followed that state, then the average will converge to the state’s value, $v_\pi(s)$, as the number of times that state is encountered approaches infinity.

If separate averages are kept for each action taken in each state, then these averages will similarly converge to the action values, $q_\pi(s,a)$. We call estimation methods of this kind _Monte Carlo methods_ because they involve averaging over many random samples of actual returns.

if there are very many states, then it may not be practical to keep separate averages for each state individually. Instead, the agent would have to maintain $v_\pi$ and $q_\pi$ as parameterized functions (with fewer parameters than states) and adjust the parameters to better match the observed returns. This can also produce accurate estimates, although much depends on the nature of the parameterized function approximator.

### Bellman equation

For any policy $\pi$ and any state $s$, the following consistency condition holds between the value of $s$ and the value of its possible successor states:

$$\begin{align}
  v_\pi(s) & \doteq \mathbb{E}_\pi\left[G_t \mid S_t = s\right] \\
  & = \mathbb{E}_\pi\left[R_{t+1} + \gamma G_{t+1} \mid S_t = s\right] \\
  & = \mathbb{E}_{\pi}\left[ R_{t+1} | S_t = s \right] + \gamma \mathbb{E}_{\pi}\left[ G_{t+1} | S_t = s \right] \\
  & = \sum_{r \in \mathcal{R}} \sum_{s' \in \mathcal{S}} \sum_{a \in \mathcal{A}} r \pi(a|s) p(s',r | a,s) + \gamma \sum_{r \in \mathcal{R}} \sum_{s' \in \mathcal{S}} \sum_{a \in \mathcal{A}} \mathbb{E}_{\pi}\left[ G_{t+1} | S_{t+1} = s' \right] p(s', r | a, s) \pi(a | s) \\
  & = \sum_{a \in \mathcal{A}} \pi(a | s)  \sum_{r \in \mathcal{R}} \sum_{s' \in \mathcal{S}} p(s', r | a, s) \left[ r + \gamma v_{\pi}(s') \right]
\end{align}$$

_Bellman equation_ for $v_\pi$ expresses a relationship between the value of a state and the values of its successor states.

Starting from state s, the root node at the top, the agent could take any of some set of actions based on its policy $\pi$. From each of these, the environment could respond with one of several next states, $s'$ (two are shown in the figure), along with a reward, $r$, depending on its dynamics given by the function p.

The _Bellman equation_ averages over all the possibilities, weighting each by its probability of occurring. It states that the value of the start state must equal the (discounted) value of the expected next state, plus the reward expected along the way.

![Back-Up diagram](https://cdn.analyticsvidhya.com/wp-content/uploads/2018/09/im_13.png)

We call diagrams like that above ***backup diagrams*** because they diagram relationships that form the basis of the update or backup operations Suppose the agent selects all four actions with equal probability in all that are at the heart of reinforcement learning methods.

## Optimal Policies and Optimal Value Functions

For finite MDPs, we can precisely define an optimal policy in the following way. Value functions define a partial ordering over policies. A policy $\pi$ is defined to be better than or equal to a policy $\pi'$ if and only if $v_\pi(s) \ge v_{\pi'}(s)$ for all $s \in \mathcal{S}$.

There is always at least one policy that is better than or equal to all other policies. This is an ***optimal policy***. Although there may be more than one, we denote all the optimal policies by $\pi^*$.

They share the same state-value function, called the ***optimal state-value function***, denoted $v^*$, and defined as
$$ v^*(s) \doteq \underset{\pi}{\mathrm{max}}\space v_\pi(s) $$
for all $s \in \mathcal{S}$.

Optimal policies also share the same ***optimal action-value*** function, denoted $q_\pi$, and defined as
$$ q^*(s,a) \doteq \underset{\pi}{\mathrm{max}}\space q_\pi(s,a) $$
for all $s \in \mathcal{S}$ and $a \in \mathcal{A}(s)$.

### Bellman optimality equation

Intuitively, the Bellman optimality equation expresses the fact that the value of a state under an optimal policy must equal the expected return for the best action from that state:
$$ 
\begin{align}
  v_*(s) & \doteq \underset{a \in \mathcal{A}(s)}{\mathrm{max}} \space q_{\pi^*}(s,a) \\
  & = \underset{a}{\mathrm{max}}\space \mathbb{E}_{\pi^*}\left[G_t \mid S_t = s, A_t = a\right] \\
  & = \underset{a}{\mathrm{max}}\space \mathbb{E}_{\pi^*}\left[R_{t+1} + \gamma G_{t+1} \mid S_t = s, A_t = a\right] \\
  & = \underset{a}{\mathrm{max}}\space \mathbb{E}\left[R_{t+1} + \gamma v^*(S_{t+1}) \mid S_t = s, A_t = a\right] \\
  & = \underset{a}{\mathrm{max}} \sum_{r \in \mathcal{R}} \sum_{s' \in \mathcal{S}} p(s', r | a, s) \left[ r + \gamma v_{\pi}(s') \right]
\end{align}
$$

The Bellman optimality equation for $q^*$ is
$$ 
\begin{align}
  q_*(s,a) & \doteq \underset{a \in \mathcal{A}(s)}{\mathrm{max}} \space q_{\pi^*}(s,a) \\
  & = \mathbb{E}\left[R_{t+1} + \gamma \underset{a'}{\mathrm{max}}\space q^*(S_{t+1},a') \mid S_t = s, A_t = a\right] \\
  & = \sum_{r \in \mathcal{R}} \sum_{s' \in \mathcal{S}} p(s', r | a, s) \left[ r + \gamma \underset{a'}{\mathrm{max}}\space q^*(s',a') \right]
\end{align}
$$

For finite MDPs, the Bellman optimality equation for $v^*$ has a unique solution. The Bellman optimality equation is actually a system of equations, one for each state, so if there are n states, then there are n equations in n unknowns.
Once one has $v^*$, it is relatively easy to determine an optimal policy. For each state $s$, there will be one or more actions at which the maximum is obtained in the Bellman optimality equation. Any policy that assigns nonzero probability only to these actions is an optimal policy. If you have the optimal value function, $v^*$, then the actions that appear best after a one-step search will be optimal actions.

Having $q^*$ makes choosing optimal actions even easier. With $q^*$, the agent does not even have to do a one-step-ahead search: for any state $s$, it can simply find any action that maximizes $q^*(s,a)$.

## Optimality and Approximation

Even if we have a complete and accurate model of the environment’s dynamics, it is usually not possible to simply compute an optimal policy by solving the Bellman optimality equation. A critical aspect of the problem facing the agent is always the computational power available to it, in particular, the amount of computation it can perform in a single time step.

A large amount of memory is often required to build up approximations of value functions, policies, and models. In tasks with small, finite state sets, it is possible to form these approximations using arrays or tables with one entry for each state (or state–action pair). This we call the ***tabular*** case, and the corresponding methods we call ***tabular methods***.

## References

+ Reinforcement Learning: An Introduction, Richard S. Sutton and Andrew G. Barto